In [2]:
import math
import numpy as np
import torch
import torch.nn as nn   

In [9]:
cmd_ids = torch.randint(low=0, high = 500, size=(10,))

env_ids = cmd_ids > ((10 / 0.02)//2)
env_ids = env_ids.nonzero().squeeze(1) 

In [13]:
is_success = 1. 
success_rate = np.random.rand(10)
is_success = is_success * (success_rate > 1.1)

In [14]:
print(is_success)
is_success = is_success.nonzero()[0]
print(is_success)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[]


In [15]:
cmd_ids[is_success] 

tensor([], dtype=torch.int64)

In [8]:
is_success.shape

(21,)